# 📘 Step 4: MLflow Evaluation - Built-in Scorers

## 목표
- MLflow의 Built-in LLM 평가 메트릭 사용
- 테스트 데이터셋 준비
- 자동 평가 실행 및 결과 분석

## 학습 포인트
- Built-in Scorer 종류와 용도
- LLM-as-Judge 개념
- Evaluation Dataset 구조
- `mlflow.genai.evaluate()` API
- 평가 결과 추적 및 비교

## 0. 사전 준비

### ⚠️ 주의사항
이 노트북을 실행하기 전에:
- **Step 0, 1, 2, 3** 완료 권장
- `.env` 파일 설정 완료
- 필요한 패키지 설치 완료

## 1. 환경 설정 및 패키지 임포트

### 1.1 환경 변수 로드

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

### 1.2 필요한 라이브러리 임포트

In [2]:
import pandas as pd
import time
from typing import Dict, Any

# MLflow
import mlflow
from mlflow import MlflowClient
from mlflow.entities import SpanType

# MLflow GenAI Evaluation
from mlflow.genai.scorers import (
    Correctness,
    RelevanceToQuery,
    Guidelines,
)
from mlflow.genai import scorer

# LangChain
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

print("✅ 라이브러리 임포트 완료")
print(f"MLflow 버전: {mlflow.__version__}")

✅ 라이브러리 임포트 완료
MLflow 버전: 3.7.0


## 2. MLflow Tracking 설정

### 2.1 Tracking URI 및 Experiment 설정

In [3]:
# Tracking URI 설정
mlflow.set_tracking_uri("./mlruns")

# Experiment 설정
experiment_name = "rag_agent_evaluation"
mlflow.set_experiment(experiment_name)

experiment = mlflow.get_experiment_by_name(experiment_name)

print("✅ MLflow 설정 완료")
print(f"📊 Experiment Name: {experiment.name}")
print(f"🆔 Experiment ID: {experiment.experiment_id}")

✅ MLflow 설정 완료
📊 Experiment Name: rag_agent_evaluation
🆔 Experiment ID: 483631771709821094


/Users/hank_kim/Documents/taehan/mlflow-genai-tutorial/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)


## 3. RAG Agent 준비

### 3.1 샘플 문서 및 Vector Store

In [4]:
# AWS 환경 변수 로드
AWS_MODEL_ID = os.getenv("AWS_MODEL_ID")
AWS_EMD_MODEL_ID = os.getenv("AWS_EMD_MODEL_ID")
AWS_REGION = os.getenv("AWS_REGION", "ap-northeast-2")

# 샘플 문서
sample_documents = [
    """
    RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.
    RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.
    이 방법은 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.
    """,
    """
    Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.
    FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다.
    Vector Store는 유사도 검색(Similarity Search)을 통해 쿼리와 가장 관련성 높은 문서를 빠르게 찾아냅니다.
    코사인 유사도나 유클리드 거리 등의 메트릭을 사용합니다.
    """,
    """
    MLflow는 기계 학습 생명주기를 관리하는 오픈소스 플랫폼입니다.
    실험 추적, 모델 버전 관리, 배포 등의 기능을 제공합니다.
    MLflow Tracing을 사용하면 LLM 애플리케이션의 실행 과정을 상세히 추적할 수 있습니다.
    """,
    """
    Evaluation은 LLM 애플리케이션의 품질을 측정하는 과정입니다.
    Faithfulness, Relevance, Correctness 등 다양한 메트릭으로 평가할 수 있습니다.
    LLM-as-Judge 방식은 다른 LLM을 사용하여 응답 품질을 자동으로 평가하는 기법입니다.
    """,
]

# Vector Store 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
documents = [Document(page_content=doc.strip()) for doc in sample_documents]
split_docs = text_splitter.split_documents(documents)

# 임베딩 및 Vector Store 생성
embeddings = BedrockEmbeddings(
    model_id=AWS_EMD_MODEL_ID,
    region_name=AWS_REGION
)

vector_store = FAISS.from_documents(split_docs, embeddings)

print(f"✅ Vector Store 생성 완료 ({len(split_docs)}개 chunks)")

✅ Vector Store 생성 완료 (4개 chunks)


### 3.2 RAG Agent 함수 정의

In [5]:
def simple_rag_agent(query: str) -> str:
    """
    간단한 RAG Agent 구현
    평가용으로 사용할 예측 함수
    """
    # 1. 문서 검색
    retrieved_docs = vector_store.similarity_search(query, k=3)
    
    # 2. Context 생성
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # 3. Prompt 생성
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "당신은 질문에 답변하는 친절한 AI 어시스턴트입니다. 제공된 컨텍스트를 기반으로 정확하고 간결한 답변을 제공하세요."),
        ("human", "컨텍스트:\n{context}\n\n질문: {query}\n\n답변:")
    ])
    
    messages = prompt_template.format_messages(context=context, query=query)
    
    # 4. LLM 호출
    llm = ChatBedrock(
        model_id=AWS_MODEL_ID,
        region_name=AWS_REGION,
        model_kwargs={"temperature": 0.7}
    )
    
    response = llm.invoke(messages)
    
    return response.content


# 테스트
test_answer = simple_rag_agent("RAG가 무엇인가요?")
print("\n✅ RAG Agent 정의 완료")
print(f"\n💬 테스트 답변:\n{test_answer}")


✅ RAG Agent 정의 완료

💬 테스트 답변:
RAG(Retrieval-Augmented Generation)는 **대규모 언어 모델(LLM)의 능력을 향상시키는 기술**입니다.

주요 특징은 다음과 같습니다:

1. **외부 지식 검색**: 외부 지식 베이스에서 관련 정보를 검색합니다.

2. **정확한 답변 생성**: 검색된 정보를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.

3. **최신 정보 활용**: 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.

즉, RAG는 LLM이 자체 학습 데이터에만 의존하지 않고, 필요할 때 외부 정보를 검색하여 더 신뢰할 수 있는 답변을 제공하는 방식입니다.


## 4. Evaluation Dataset 준비

### 4.1 Dataset 구조 이해

In [6]:
print("\n" + "="*80)
print("📚 Evaluation Dataset 구조")
print("="*80)

print("""
MLflow의 Evaluation Dataset은 다음 컬럼을 포함할 수 있습니다:

1. inputs (필수):
   - 모델에 입력할 데이터
   - Dict 형태: {"query": "질문"} 또는 {"question": "질문"}
   - predict_fn의 인자와 매칭됨

2. expectations (선택, 권장):
   - Ground truth 또는 기대 결과
   - Correctness 등 비교 평가에 사용
   - Dict 형태: {"expected_response": "정답"}

3. outputs (선택):
   - 이미 생성된 응답 (predict_fn 없이 평가 시)
   - Dict 또는 String

4. 기타 메타데이터 (선택):
   - 평가에 필요한 추가 정보
   - 예: {"retrieved_documents": [...]}
""")

print("="*80)


📚 Evaluation Dataset 구조

MLflow의 Evaluation Dataset은 다음 컬럼을 포함할 수 있습니다:

1. inputs (필수):
   - 모델에 입력할 데이터
   - Dict 형태: {"query": "질문"} 또는 {"question": "질문"}
   - predict_fn의 인자와 매칭됨

2. expectations (선택, 권장):
   - Ground truth 또는 기대 결과
   - Correctness 등 비교 평가에 사용
   - Dict 형태: {"expected_response": "정답"}

3. outputs (선택):
   - 이미 생성된 응답 (predict_fn 없이 평가 시)
   - Dict 또는 String

4. 기타 메타데이터 (선택):
   - 평가에 필요한 추가 정보
   - 예: {"retrieved_documents": [...]}



### 4.2 Evaluation Dataset 생성

In [7]:
# Evaluation Dataset 정의
eval_data = [
    {
        "inputs": {"query": "RAG가 무엇인가요?"},
        "expectations": {
            "expected_response": "RAG는 Retrieval-Augmented Generation의 약자로, 외부 지식 베이스에서 관련 정보를 검색하여 LLM의 답변 생성 능력을 향상시키는 기술입니다."
        }
    },
    {
        "inputs": {"query": "Vector Store는 어떻게 작동하나요?"},
        "expectations": {
            "expected_response": "Vector Store는 문서를 임베딩 벡터로 변환하여 저장하고, 쿼리와의 유사도를 계산하여 관련 문서를 검색하는 데이터베이스입니다."
        }
    },
    {
        "inputs": {"query": "MLflow Tracing이 무엇인가요?"},
        "expectations": {
            "expected_response": "MLflow Tracing은 LLM 애플리케이션의 실행 과정을 Span 단위로 추적하여 성능과 동작을 분석할 수 있게 해주는 기능입니다."
        }
    },
    {
        "inputs": {"query": "LLM-as-Judge란 무엇인가요?"},
        "expectations": {
            "expected_response": "LLM-as-Judge는 다른 LLM을 활용하여 AI 응답의 품질을 자동으로 평가하는 기법으로, Faithfulness, Relevance 등의 메트릭을 평가합니다."
        }
    },
]

# Pandas DataFrame으로 변환
eval_dataset = pd.DataFrame(eval_data)

print("\n✅ Evaluation Dataset 생성 완료")
print(f"\n📊 Dataset 크기: {len(eval_dataset)}개")
print("\n" + "="*80)
print("Dataset 미리보기:")
print("="*80)
print(eval_dataset[['inputs']].head())


✅ Evaluation Dataset 생성 완료

📊 Dataset 크기: 4개

Dataset 미리보기:
                                  inputs
0               {'query': 'RAG가 무엇인가요?'}
1  {'query': 'Vector Store는 어떻게 작동하나요?'}
2    {'query': 'MLflow Tracing이 무엇인가요?'}
3      {'query': 'LLM-as-Judge란 무엇인가요?'}


## 5. Built-in Scorers 이해

### 5.1 주요 Built-in Scorers

In [8]:
print("\n" + "="*80)
print("🎯 MLflow Built-in Scorers")
print("="*80)

scorers_info = [
    ["Scorer", "설명", "입력", "출력"],
    ["-" * 20, "-" * 40, "-" * 20, "-" * 15],
    [
        "Correctness",
        "예상 답변과 실제 답변 비교",
        "outputs, expectations",
        "0-1 점수"
    ],
    [
        "RelevanceToQuery",
        "답변이 질문과 관련있는지 평가",
        "inputs, outputs",
        "0-1 점수"
    ],
    [
        "Guidelines",
        "커스텀 가이드라인 준수 여부",
        "outputs, guidelines",
        "Boolean/점수"
    ],
    [
        "Safety",
        "유해 콘텐츠 감지",
        "outputs",
        "Boolean"
    ],
    [
        "RetrievalGroundedness",
        "검색 문서 기반 답변 생성 확인",
        "outputs, retrieved_docs",
        "0-1 점수"
    ],
]

for row in scorers_info:
    print(f"{row[0]:<20} | {row[1]:<40} | {row[2]:<20} | {row[3]:<15}")

print("\n" + "="*80)
print("💡 모든 Scorer는 LLM-as-Judge 방식을 사용합니다.")
print("   기본 Judge Model: gpt-4o-mini (변경 가능)")
print("="*80)


🎯 MLflow Built-in Scorers
Scorer               | 설명                                       | 입력                   | 출력             
-------------------- | ---------------------------------------- | -------------------- | ---------------
Correctness          | 예상 답변과 실제 답변 비교                          | outputs, expectations | 0-1 점수         
RelevanceToQuery     | 답변이 질문과 관련있는지 평가                         | inputs, outputs      | 0-1 점수         
Guidelines           | 커스텀 가이드라인 준수 여부                          | outputs, guidelines  | Boolean/점수     
Safety               | 유해 콘텐츠 감지                                | outputs              | Boolean        
RetrievalGroundedness | 검색 문서 기반 답변 생성 확인                        | outputs, retrieved_docs | 0-1 점수         

💡 모든 Scorer는 LLM-as-Judge 방식을 사용합니다.
   기본 Judge Model: gpt-4o-mini (변경 가능)


### 5.2 Scorer 초기화

In [9]:
# Built-in Scorers 정의
scorers = [
    Correctness(
        name="correctness",
        model=f"bedrock:/{AWS_MODEL_ID}",  # Judge LLM 지정
    ),
    RelevanceToQuery(
        name="relevance",
        model=f"bedrock:/{AWS_MODEL_ID}",
    ),
    Guidelines(
        name="is_korean",
        guidelines="답변은 한국어로 작성되어야 합니다.",
        model=f"bedrock:/{AWS_MODEL_ID}",
    ),
]

print("\n✅ Scorers 초기화 완료")
print(f"\n📊 총 {len(scorers)}개의 Scorer 정의:")
for _scorer in scorers:
    print(f"  • {_scorer.name}")


✅ Scorers 초기화 완료

📊 총 3개의 Scorer 정의:
  • correctness
  • relevance
  • is_korean


## 6. 평가 실행

### 6.1 mlflow.genai.evaluate() 사용

In [ ]:
print("\n" + "="*80)
print("🚀 평가 실행 시작")
print("="*80)

# predict_fn 래핑 (mlflow.genai.evaluate가 기대하는 형식)
def predict_fn(*, query: str) -> str:
    """
    MLflow evaluation을 위한 예측 함수
    
    중요: MLflow는 inputs dict를 키워드 인자로 언팩하므로
    키워드 전용 인자(*)로 정의해야 함
    inputs={"query": "..."} -> predict_fn(query="...")
    """
    return simple_rag_agent(query)

# 평가 실행
results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=scorers,
)

print("\n✅ 평가 완료")
print("="*80)

### 6.2 평가 결과 확인

In [13]:
print("\n" + "="*80)
print("📊 평가 메트릭 요약")
print("="*80)

# 전체 메트릭
metrics = results.metrics

print("\n📈 집계 메트릭:")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.3f}")
    else:
        print(f"  • {key}: {value}")

print("\n" + "="*80)


📊 평가 메트릭 요약

📈 집계 메트릭:
  • relevance/mean: 1.000
  • correctness/mean: 0.750
  • is_korean/mean: 1.000



### 6.3 상세 평가 결과 테이블

In [14]:
results.result_df.head()

,trace_id,correctness/value,is_korean/value,expected_response/value,relevance/value,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-ae4b37d361a68c28543abb3e6c76aab2,yes,yes,"RAG는 Retrieval-Augmented Generation의 약자로, 외부 지...",yes,"{""info"": {""trace_id"": ""tr-ae4b37d361a68c28543a...",None,OK,1766024892010,4317,"{'body': '{""anthropic_version"": ""bedrock-2023-...",{'ResponseMetadata': {'RequestId': 'da15a14b-6...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.eval.requestId': '21584ef0-3728-42e5-...,"[{'trace_id': 'rks302GmjChUOrs+bHaqsg==', 'spa...",[{'assessment_id': 'a-b176d0e6eba14aaea99b435d...
1,tr-929500295e96c472e96cd58d1f9f85f5,yes,yes,LLM-as-Judge는 다른 LLM을 활용하여 AI 응답의 품질을 자동으로 평가하...,yes,"{""info"": {""trace_id"": ""tr-929500295e96c472e96c...",None,OK,1766024892118,3288,"{'body': '{""anthropic_version"": ""bedrock-2023-...",{'ResponseMetadata': {'RequestId': '8f0c971d-8...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.eval.requestId': '6be5f006-4518-4cb9-...,"[{'trace_id': 'kpUAKV6WxHLpbNWNH5+F9Q==', 'spa...",[{'assessment_id': 'a-7df6c119b53646a293740eb4...
2,tr-fc9420d3a4c7d7656b7503ff34c8a7db,no,yes,MLflow Tracing은 LLM 애플리케이션의 실행 과정을 Span 단위로 추적...,yes,"{""info"": {""trace_id"": ""tr-fc9420d3a4c7d7656b75...",None,OK,1766024892119,3628,"{'body': '{""anthropic_version"": ""bedrock-2023-...",{'ResponseMetadata': {'RequestId': 'e776bf45-b...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.eval.requestId': 'cf4dd19e-401d-4ee1-...,"[{'trace_id': '/JQg06TH12VrdQP/NMin2w==', 'spa...",[{'assessment_id': 'a-1a44b89488a243a6b30df3ee...
3,tr-a7dc77aebf7f65db231d7f8e7a5f717f,yes,yes,"Vector Store는 문서를 임베딩 벡터로 변환하여 저장하고, 쿼리와의 유사도를...",yes,"{""info"": {""trace_id"": ""tr-a7dc77aebf7f65db231d...",None,OK,1766024892116,6242,"{'body': '{""anthropic_version"": ""bedrock-2023-...",{'ResponseMetadata': {'RequestId': 'f405a8ee-a...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.eval.requestId': 'd3b07595-8c25-41f2-...,"[{'trace_id': 'p9x3rr9/ZdsjHX+Oel9xfw==', 'spa...",[{'assessment_id': 'a-50cc65ca096248a4b26c7eb4...


In [ ]:
# request 컬럼 구조 확인
import json

eval_results_table = results.result_df

print("\\n" + "="*80)
print("🔍 Request/Response 컬럼 구조 분석")
print("="*80)

# 첫 번째 row의 request 구조 확인
first_request = eval_results_table['request'][0]
first_response = eval_results_table['response'][0]

print("\\n📝 Request 구조:")
print(f"  타입: {type(first_request)}")
print(f"  키: {first_request.keys() if isinstance(first_request, dict) else 'N/A'}")

print("\\n📝 Response 구조:")
print(f"  타입: {type(first_response)}")
print(f"  키: {first_response.keys() if isinstance(first_response, dict) else 'N/A'}")

# body가 JSON 문자열이므로 파싱
if 'body' in first_request:
    body_parsed = json.loads(first_request['body'])
    print("\\n📄 Request Body 내용 (파싱됨):")
    print(f"  - messages 타입: {type(body_parsed.get('messages'))}")
    print(f"  - messages 개수: {len(body_parsed.get('messages', []))}")
    if body_parsed.get('messages'):
        print(f"  - 첫 메시지: {body_parsed['messages'][0]['role']}")
        
print("\\n" + "="*80)

In [ ]:
# 상세 결과 테이블
import json

df = results.result_df.copy()

def extract_input_from_trace(row):
    """
    Trace의 spans에서 실제 입력 추출
    """
    try:
        # spans 컬럼에서 정보 추출
        if 'spans' in row and row['spans']:
            # 첫 번째 span의 inputs 확인
            first_span = row['spans'][0]
            if 'inputs' in first_span:
                inputs = first_span.get('inputs', {})
                return inputs.get('query', '')
        
        # 대안: request body에서 추출
        if isinstance(row['request'], dict) and 'body' in row['request']:
            body = json.loads(row['request']['body'])
            messages = body.get('messages', [])
            if messages:
                content = messages[0].get('content', '')
                # "질문: xxx" 패턴에서 추출
                if '질문:' in content:
                    return content.split('질문:', 1)[1].split('답변:', 1)[0].strip()
        return ''
    except Exception as e:
        return f"Error: {str(e)}"

def extract_output_from_trace(row):
    """
    Trace의 spans 또는 response에서 실제 출력 추출
    """
    try:
        # spans 컬럼에서 outputs 추출
        if 'spans' in row and row['spans']:
            first_span = row['spans'][0]
            if 'outputs' in first_span:
                outputs = first_span.get('outputs')
                # outputs가 문자열이면 그대로 반환
                if isinstance(outputs, str):
                    return outputs
                # dict면 적절한 키 찾기
                elif isinstance(outputs, dict):
                    return outputs.get('content', outputs.get('text', str(outputs)))
        
        # 대안: response 컬럼에서 직접 추출 (Bedrock 응답 구조)
        if isinstance(row['response'], dict):
            # Bedrock 응답 구조: output.message.content[0].text
            if 'output' in row['response']:
                output = row['response']['output']
                if 'message' in output:
                    content = output['message'].get('content', [])
                    if content and len(content) > 0:
                        return content[0].get('text', '')
        return ''
    except Exception as e:
        return f"Error: {str(e)}"

# 컬럼 추가
df["input"] = df.apply(extract_input_from_trace, axis=1)
df["output"] = df.apply(extract_output_from_trace, axis=1)

print("\\n" + "="*80)
print("📋 각 샘플별 평가 결과")
print("="*80)

# 주요 컬럼만 출력
display_columns = [
    col for col in df.columns 
    if any(x in col for x in ['input', 'output', 'correctness', 'relevance', 'is_korean'])
]

# 간단한 미리보기
for idx, row in df.iterrows():
    print(f"\\n샘플 {idx + 1}:")
    print(f"  질문: {row['input'][:80]}...")
    print(f"  답변: {row['output'][:80]}...")
    print(f"  Correctness: {row.get('correctness/value', 'N/A')}")
    print(f"  Relevance: {row.get('relevance/value', 'N/A')}")
    print(f"  Is Korean: {row.get('is_korean/value', 'N/A')}")

print("\\n" + "="*80)

### 6.4 개별 샘플 상세 분석

In [105]:
# 첫 번째 샘플 상세 보기
first_row = df.iloc[0]

print("\n" + "="*80)
print("🔍 첫 번째 샘플 상세 분석")
print("="*80)

print(f"\n❓ 질문: {first_row['input']}")
print(f"\n💬 생성된 답변:\n{first_row['output']}")
print(f"\n✅ 기대 답변:\n{first_row['expected_response/value']}")

print("\n" + "-" * 80)
print("📊 평가 점수:")
print("-" * 80)

# Scorer별 점수 및 Rationale
for scorer_name in ['correctness', 'relevance', 'is_korean']:
    score_col = f"{scorer_name}/value"
    # rationale_col = f"{scorer_name}/rationale"
    
    if score_col in first_row:
        score = first_row[score_col]
        # rationale = first_row.get(rationale_col, "N/A")
        
        print(f"\n{scorer_name.upper()}:")
        print(f"  점수: {score}")
        # print(f"  근거: {rationale}")

print("\n" + "="*80)


🔍 첫 번째 샘플 상세 분석

❓ 질문: MLflow Tracing이 무엇인가요?

💬 생성된 답변:
MLflow Tracing은 **LLM 애플리케이션의 실행 과정을 상세히 추적할 수 있는 기능**입니다.

MLflow는 기계 학습 생명주기를 관리하는 오픈소스 플랫폼으로, 실험 추적, 모델 버전 관리, 배포 등의 다양한 기능을 제공하는데, 그 중 MLflow Tracing은 특히 LLM(대규모 언어 모델) 애플리케이션의 동작을 모니터링하고 디버깅하는 데 유용한 도구입니다.

✅ 기대 답변:
MLflow Tracing은 LLM 애플리케이션의 실행 과정을 Span 단위로 추적하여 성능과 동작을 분석할 수 있게 해주는 기능입니다.

--------------------------------------------------------------------------------
📊 평가 점수:
--------------------------------------------------------------------------------

CORRECTNESS:
  점수: no

RELEVANCE:
  점수: yes

IS_KOREAN:
  점수: yes



## 7. 커스텀 Scorer 추가

### 7.1 간단한 커스텀 Scorer 정의

In [25]:
from mlflow.entities import Feedback
from mlflow.genai import scorer

@scorer
def answer_length(outputs: str) -> Feedback:
    length = len(outputs)

    if length < 50:
        value = "too_short"
        rationale = f"답변이 너무 짧습니다 ({length}자)."
    elif length > 500:
        value = "too_long"
        rationale = f"답변이 너무 깁니다 ({length}자)."
    else:
        value = "appropriate"
        rationale = f"답변 길이가 적절합니다 ({length}자)."

    return Feedback(
        value=value,
        rationale=rationale,
        metadata={"length": length},
    )


@scorer
def contains_keywords(outputs: str, inputs: Dict[str, Any]) -> bool:
    """
    답변에 질문의 핵심 키워드가 포함되어 있는지 확인
    """
    query = inputs.get("query", "").lower()
    outputs_lower = outputs.lower()
    
    # 간단한 키워드 추출 (실제로는 더 정교한 방법 사용)
    keywords = [word for word in query.split() if len(word) > 2]
    
    # 키워드 중 하나라도 답변에 포함되어 있는지
    return any(keyword in outputs_lower for keyword in keywords)


print("✅ 커스텀 Scorer 정의 완료")
print("  • answer_length: 답변 길이 평가")
print("  • contains_keywords: 키워드 포함 여부")

✅ 커스텀 Scorer 정의 완료
  • answer_length: 답변 길이 평가
  • contains_keywords: 키워드 포함 여부


### 7.2 커스텀 Scorer 포함하여 재평가

In [ ]:
print("\n" + "="*80)
print("🔄 커스텀 Scorer 포함 재평가")
print("="*80)

# 모든 Scorer 결합
all_scorers = scorers + [answer_length, contains_keywords]

# predict_fn도 동일하게 키워드 인자로 정의되어 있어야 함
def predict_fn_with_custom(*, query: str) -> str:
    """키워드 전용 인자로 정의된 predict 함수"""
    return simple_rag_agent(query)

# 재평가
results_with_custom = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn_with_custom,
    scorers=all_scorers,
)

print("\n✅ 재평가 완료")

# 새로운 메트릭 확인
print("\n📊 업데이트된 메트릭:")
for key, value in results_with_custom.metrics.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.3f}")
    else:
        print(f"  • {key}: {value}")

print("\n" + "="*80)

## 8. 평가 결과 비교

### 8.1 여러 Run 비교

In [21]:
# 실험의 모든 Run 검색
client = MlflowClient()
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["start_time DESC"],
    max_results=10
)

print("\n" + "="*80)
print("📊 Evaluation Runs 비교")
print("="*80)

print(f"\n{'Run ID':<35}{'Correctness':<15}")
print("-" * 80)

for run in runs[:5]:
    run_id = run.info.run_id[:32]
    correctness = run.data.metrics.get('correctness/mean', 0)
    
    print(f"{run_id:<35} {correctness:>12.3f}")

print("\n" + "="*80)


📊 Evaluation Runs 비교

Run ID                             Correctness    
--------------------------------------------------------------------------------
394cb8dcedd44763acadf89a2edab488           0.750
3fde436d0c354ce8995fea5e80052ca3           0.000
a712e041bff84feeba7ddcb7317a5019           0.750
70b656ca03f3443a8cb0016afad7c27f           0.750
28427d545efb458fb8d5fe548bf169c9           0.000



## 9. MLflow UI에서 결과 확인

### 9.1 UI 탐색 가이드

In [22]:
print("\n" + "="*80)
print("🌐 MLflow UI에서 평가 결과 확인하기")
print("="*80)

print("""
1️⃣ MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ Experiment 선택:
   • 'rag_agent_evaluation' 실험 클릭

3️⃣ Evaluation Runs 확인:
   • 각 Run은 하나의 평가 실행을 나타냄
   • Metrics 컬럼에서 집계 점수 확인
   • Run 이름으로 model_id 확인

4️⃣ 개별 Run 상세 보기:
   • Run 클릭
   • "Evaluation" 탭으로 이동
   • 각 샘플별 입력, 출력, 점수 확인

5️⃣ Scorer별 상세 분석:
   • Rationale 확인 (LLM이 평가한 근거)
   • 점수 분포 시각화
   • 실패한 샘플 필터링

6️⃣ Run 비교:
   • 여러 Run 선택 (체크박스)
   • "Compare" 버튼 클릭
   • Metrics 차트로 비교

7️⃣ Trace 확인 (자동 생성):
   • 각 평가 샘플마다 Trace 생성됨
   • "Traces" 탭에서 실행 과정 확인
""")

print("="*80)


🌐 MLflow UI에서 평가 결과 확인하기

1️⃣ MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ Experiment 선택:
   • 'rag_agent_evaluation' 실험 클릭

3️⃣ Evaluation Runs 확인:
   • 각 Run은 하나의 평가 실행을 나타냄
   • Metrics 컬럼에서 집계 점수 확인
   • Run 이름으로 model_id 확인

4️⃣ 개별 Run 상세 보기:
   • Run 클릭
   • "Evaluation" 탭으로 이동
   • 각 샘플별 입력, 출력, 점수 확인

5️⃣ Scorer별 상세 분석:
   • Rationale 확인 (LLM이 평가한 근거)
   • 점수 분포 시각화
   • 실패한 샘플 필터링

6️⃣ Run 비교:
   • 여러 Run 선택 (체크박스)
   • "Compare" 버튼 클릭
   • Metrics 차트로 비교

7️⃣ Trace 확인 (자동 생성):
   • 각 평가 샘플마다 Trace 생성됨
   • "Traces" 탭에서 실행 과정 확인



## 10. 고급: 사전 생성된 출력으로 평가

### 10.1 predict_fn 없이 평가

In [27]:
# 사전에 생성된 응답을 포함한 데이터셋
pregenerated_data = [
    {
        "inputs": {"query": "RAG가 무엇인가요?"},
        "outputs": "RAG는 Retrieval-Augmented Generation의 약자입니다.",
        "expectations": {
            "expected_response": "RAG는 검색 증강 생성 기술입니다."
        }
    },
    {
        "inputs": {"query": "Vector Store는 무엇인가요?"},
        "outputs": "Vector Store는 임베딩 벡터를 저장하는 데이터베이스입니다.",
        "expectations": {
            "expected_response": "Vector Store는 벡터 데이터베이스입니다."
        }
    },
]

pregenerated_df = pd.DataFrame(pregenerated_data)

# predict_fn 없이 평가
results_pregenerated = mlflow.genai.evaluate(
    data=pregenerated_df,
    # predict_fn 없음 - outputs 컬럼 사용
    scorers=scorers,
)

print("\n✅ 사전 생성 출력 평가 완료")
print("\n📊 메트릭:")
for key, value in results_pregenerated.metrics.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.3f}")
    else:
        print(f"  • {key}: {value}")


✨ Evaluation completed.

Metrics and evaluation results are logged to the MLflow run:
  Run name: orderly-squirrel-762
  Run ID: 4b97c619ce8e44bfabfd67d801442cbe

To view the detailed evaluation results with sample-wise scores,
open the Traces tab in the Run page in the MLflow UI.


✅ 사전 생성 출력 평가 완료

📊 메트릭:
  • relevance/mean: 1.000
  • is_korean/mean: 1.000
  • correctness/mean: 0.500


In [29]:
results_pregenerated.result_df.head()

,trace_id,is_korean/value,correctness/value,relevance/value,expected_response/value,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-1960a3671341e6766fefce44b1f65edb,yes,yes,yes,Vector Store는 벡터 데이터베이스입니다.,"{""info"": {""trace_id"": ""tr-1960a3671341e6766fef...",None,OK,1766023807251,0,{'query': 'Vector Store는 무엇인가요?'},Vector Store는 임베딩 벡터를 저장하는 데이터베이스입니다.,"{'mlflow.trace_schema.version': '3', 'mlflow.u...",{'mlflow.eval.requestId': '2194d9883c7dc13237d...,"[{'trace_id': 'GWCjZxNB5nZv785EsfZe2w==', 'spa...",[{'assessment_id': 'a-343adb660fa74edb802a86d9...
1,tr-1ed92af383eceea8b8a145f38f705748,yes,no,yes,RAG는 검색 증강 생성 기술입니다.,"{""info"": {""trace_id"": ""tr-1ed92af383eceea8b8a1...",None,OK,1766023807250,0,{'query': 'RAG가 무엇인가요?'},RAG는 Retrieval-Augmented Generation의 약자입니다.,"{'mlflow.trace_schema.version': '3', 'mlflow.u...",{'mlflow.eval.requestId': '161a7107e3c32ee5c96...,"[{'trace_id': 'Htkq84Ps7qi4oUXzj3BXSA==', 'spa...",[{'assessment_id': 'a-bb7737eaec55403297510906...


## 11. Best Practices

### 11.1 평가 모범 사례

In [ ]:
print("\n" + "="*80)
print("💡 MLflow Evaluation Best Practices")
print("="*80)

print("""
1. Dataset 설계:
   ✅ 다양한 쿼리 타입 포함 (간단한 질문, 복잡한 질문)
   ✅ Edge case 포함 (모호한 질문, 잘못된 질문)
   ✅ Ground truth 품질 확보 (전문가 검토)
   ✅ 최소 20-50개 샘플 (통계적 유의성)

2. Scorer 선택:
   ✅ 목적에 맞는 Scorer 조합
   ✅ Correctness + Relevance는 기본
   ✅ 도메인별 커스텀 Scorer 추가
   ✅ Guidelines로 비즈니스 규칙 검증

3. Judge Model 선택:
   ✅ gpt-4o-mini: 빠르고 저렴 (기본 권장)
   ✅ gpt-4o: 더 정확한 평가 필요 시
   ✅ claude-opus-4: 대안 LLM

4. 평가 실행:
   ✅ 여러 버전으로 A/B 테스트
   ✅ model_id로 명확한 버전 관리
   ✅ 정기적인 재평가 (데이터 변경 시)

5. 결과 분석:
   ✅ 낮은 점수 샘플 집중 분석
   ✅ Rationale 검토 (왜 실패했는지)
   ✅ Trace 확인 (실행 과정 디버깅)
   ✅ 패턴 발견 및 개선

6. 비용 최적화:
   ✅ 샘플 수 적절히 조절
   ✅ Judge model 선택 (mini vs full)
   ✅ 캐싱 활용 (동일 입력 재평가)
""")

print("="*80)

## 12. 요약 및 다음 단계

### ✅ 완료한 작업
1. Evaluation Dataset 준비 및 구조 이해
2. Built-in Scorers 사용 (Correctness, RelevanceToQuery, Guidelines)
3. mlflow.genai.evaluate() API로 자동 평가 실행
4. 커스텀 Scorer 정의 및 추가
5. 평가 결과 분석 및 비교
6. MLflow UI에서 결과 확인 방법 학습

### 📊 주요 학습 내용
- **Evaluation Dataset**: inputs, expectations, outputs 구조
- **Built-in Scorers**: LLM-as-Judge 방식의 자동 평가
- **mlflow.genai.evaluate()**: 평가 실행 API
- **Feedback**: Scorer가 반환하는 평가 결과 (값, 근거, 메타데이터)
- **model_id**: 버전 관리 및 비교를 위한 식별자

### 🎯 다음 단계 (Step 5)
다음 노트북에서는 **커스텀 RAG Scorers**를 만들어:
- RAG 특화 메트릭 정의 (precision, recall, diversity)
- Citation 품질 평가
- Trace 기반 평가
- 프로덕션 모니터링 준비

→ `05_mlflow_evaluation_custom_rag_scorers.ipynb`로 계속하세요!

### 💡 Built-in Scorers의 장점
1. **Zero-code 평가**: 즉시 사용 가능한 검증된 메트릭
2. **LLM-as-Judge**: 사람의 판단과 유사한 평가
3. **자동화**: 대규모 평가 가능
4. **추적 가능**: MLflow에 모든 결과 저장
5. **비교 가능**: 여러 버전 간 객관적 비교

## 📚 참고 자료

- [MLflow GenAI Evaluation Guide](https://mlflow.org/docs/latest/genai/eval-monitor/)
- [Built-in Scorers Documentation](https://mlflow.org/docs/latest/genai/concepts/scorers/)
- [mlflow.genai.evaluate() API](https://mlflow.org/docs/latest/python_api/mlflow.genai.html#mlflow.genai.evaluate)
- [Evaluation Quickstart](https://mlflow.org/docs/latest/genai/eval-monitor/quickstart/)
- [LLM-as-Judge Best Practices](https://arxiv.org/abs/2306.05685)